In [104]:
import pandas as pd
import numpy as np
import transformers
from transformers import (
    pipeline,
    TextClassificationPipeline,
    AutoTokenizer
)
import torch

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((0, 1),clip=True)
scaler.fit([[0], [100]])
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)
model_name = "distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

MAX_LEN = 512
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 3e-05


cuda


In [105]:
data = pd.read_parquet("test_drob_ner_nolora.gzip")

In [107]:
# model_path = "./drob-ner-nolora-output/"
model_path = "./drob-ner-nolora-output/checkpoint-19260"
id2label = {0:0, 1:1, 2:2}
label2id = {v:k for i,(k, v) in enumerate(id2label.items())}
model_c = transformers.AutoModelForTokenClassification.from_pretrained(model_path, 
                                                                        num_labels=3,
                                                                        id2label=id2label,
                                                                        label2id=label2id)
model_c.to(device)

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Lay

In [9]:
data.iloc[0]['sentences']
data.iloc[0]['tag']


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int64)

In [4]:
df1 = data.sample(1)
df1

,sentences,tag
56694,"Combine flour, spices, baking powder and salt....","[0, 1, 1, 1, 2, 0, 1, 0, 0, 0, 1, 2, 0, 1, 0, ..."


In [108]:
def tag_tokens(original, ids, og_tag):
    tokens = (tokenizer.convert_ids_to_tokens([int(i) for i in ids]))
    tagt = []
    tsplit = original.split()
    sent_ind = 0
    token_ind = 0
    puncs = ';:.,()-"\''
    # tag tokens
    while sent_ind < (len(tsplit)):
        while token_ind < len(tokens):
            if tokens[token_ind]=='<s>':
                token_ind += 1
                continue
            temp_t = tokens[token_ind]
            temp_t = temp_t.replace('Ġ', '')
            if temp_t in puncs:
                tagt.append(0)
                token_ind += 1
            elif temp_t in tsplit[sent_ind]:
                tagt.append(og_tag[sent_ind])
                token_ind += 1
            else:
                break
        sent_ind += 1
    # tagt += [0]
    # tagt += [0]*(MAX_LEN-len(tagt))
    # print(len(tagt))
    # print(len(tokens), len(tagt))
    # for i in range(len(tokens)):
    #     print(tokens[i], tagt[i])
    return tagt
def tok_align(data):
    labels = []
    encodings = tokenizer.batch_encode_plus(
                    data['sentences'].tolist(),
                    add_special_tokens=True,
                    max_length=MAX_LEN,
                    # padding='max_length',
                    return_token_type_ids=True,
                    truncation=True
                )
    for i in range(len(data)):
        # print(data.tag.iloc[i])
        labels.append(tag_tokens(data.sentences.iloc[i], encodings['input_ids'][i], data.tag.iloc[i]))
    return encodings, labels

In [109]:
t1 = tok_align(df1)[1]

In [14]:
len(t1)

1

In [130]:
# class logitpipeline(TextClassificationPipeline):
#     def postprocess(self, model_outputs):
#         best_class = model_outputs["logits"]
#         return best_class
pipe = pipeline(task = 'ner', model=model_c, tokenizer=tokenizer, device=0)

In [135]:
import re
model_name = "distilroberta-base"
id2label = {0:'O', 1:'B-food', 2:'I-food'}
label2id = {v:k for i,(k, v) in enumerate(id2label.items())}
model_path = "./drob-ner-nolora-output/checkpoint-19260/"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_c = transformers.AutoModelForTokenClassification.from_pretrained(model_path, 
                                                            num_labels=3,
                                                            id2label=id2label,
                                                            label2id=label2id)
pipe = pipeline(task="ner", model=model_c.to("cpu"), binary_output=True, tokenizer=tokenizer, aggregation_strategy='first')
def extract_food(test_sent):
    ret = []
    out = pipe(test_sent)
    temp = ""
    last_end = 0
    print(out)
    for i in out:
        if i['start']-1 == last_end or i['start'] == last_end:
            temp += i['word']
            last_end = i['end']
        else:
            ret.append(temp.replace('tbsps','').strip().lower())
            last_end = i['end']
            temp = i['word']
        # if i['entity'] != "none":
        #     temp += (i['word'])
        # elif temp and  "-" in i['word']:
        #     temp += (i['word'])
        # elif temp and len(temp) == 1:
        #     temp = ""
        # elif temp and len(temp) > 1:
        #     ret.append(temp.replace("Ġ", " ").strip().lower())
        #     temp = ""
    if temp and len(temp) > 1:
        ret.append(temp.replace("Ġ", " ").strip().lower())
    final_ret = []
    ret = list(dict.fromkeys(ret))
    for i in ret:
        switch = False
        temp_ret = list(filter(lambda x:x!=i, ret))
        for j in temp_ret:
            if i in j:
                switch = True
                break
        if switch:
            switch = False
            continue
        final_ret.extend(re.split('[,;.!]', i))
    print(final_ret)
    final_ret = [i.strip() for i in final_ret if i and i!=' ']
    return final_ret

In [136]:
extract_food('In a large pot of salted boiling water, cook pasta according to package directions until al dente. Drain, reserving 1 cup pasta water, and return to pot.')


[{'entity_group': 'food', 'score': 0.99999225, 'word': ' water,', 'start': 33, 'end': 39}, {'entity_group': 'food', 'score': 0.9999939, 'word': ' pasta', 'start': 45, 'end': 50}, {'entity_group': 'food', 'score': 0.99957144, 'word': ' al dente.', 'start': 89, 'end': 98}, {'entity_group': 'food', 'score': 0.99998623, 'word': ' pasta water,', 'start': 122, 'end': 134}]
['al dente', '', 'pasta water', '']


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\pipelines\token_classification.py:393: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


['al dente', 'pasta water']

In [68]:
df1 = pd.DataFrame(data.iloc[153]).T

In [24]:
df1

,sentences,tag
19380,"In bottom of 13 x 9 x 2-inch baking dish, spre...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [69]:
p1 = pipe(df1.iloc[0]['sentences'])
print([i['entity'] for i in p1])
print(tok_align(df1)[1][0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [112]:
enc_labels = tok_align(data)[1]

In [113]:
ingr = data.sentences.tolist()

In [114]:
def data():
    for i in range(len(ingr)):
        yield ingr[i]

In [115]:
output = []
count = 0
# for s in sents:
#     count += 1
#     if count % 100 ==0:
#         print(count)
#     output.append(pipe(s))
for out in pipe(data()):
    count += 1
    if count % 100 ==0:
        print(count, end='\r')
    output.append(out)

In [116]:
newoutput = [[i['entity'] for i in j] for j in output]

In [95]:
enc_labels[1]

[0, 0, 0, 0, 0, 0, 1, 0]

In [117]:
for i in range(len(newoutput)):
    if len(newoutput[i]) != len(enc_labels[i]):
        print(newoutput[i])
        print(enc_labels[i])
        raise RuntimeError

In [118]:
out_df = pd.DataFrame({"output":newoutput, "labels":enc_labels})

In [119]:
out_df.to_parquet("pred_drob_nolora.gzip", compression='gzip')

In [100]:
out_df

,output,labels
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 1, 0]"
2,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
3,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 0, 0, 0]"
...,...,...
7703,"[0, 0, 0, 0, 1, 0, 0, 1, 2, 2, 2, 0]","[0, 0, 0, 0, 1, 0, 0, 1, 2, 2, 2, 0]"
7704,"[0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, ...","[0, 1, 2, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, ..."
7705,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
7706,"[0, 1, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [25]:
# encodings = tokenizer.batch_encode_plus(
#                     data['sentences'].tolist(),
#                     add_special_tokens=True,
#                     max_length=MAX_LEN,
#                     padding='max_length',
#                     return_token_type_ids=True,
#                     truncation=True,
#                     return_tensors="pt"
#                 )

In [7]:
# # def enc(sent)
# encodings = tokenizer.encode_plus(
#                     data['sentences'].tolist()[0],
#                     add_special_tokens=True,
#                     max_length=MAX_LEN,
#                     padding='max_length',
#                     return_token_type_ids=True,
#                     truncation=True,
#                     return_tensors="pt"
#                 )
# ret = [val.clone().detach().to("cuda") for val in encodings.values()]
# ret

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [5]:
# class cooking_time_model(torch.nn.Module):
#     def __init__(self):
#         super(cooking_time_model, self).__init__()
#         self.l1 = transformers.RobertaModel.from_pretrained(pretrained)
#         self.l2 = torch.nn.Dropout(0.3)
#         self.l3 = torch.nn.Linear(768, MAX_LEN)

#     def forward(self, ids, mask, token_type_ids):
#         output_1 = self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
#         output_2 = self.l2(output_1.pooler_output)
#         output = self.l3(output_2)
#         return output
model_path = "./drob-mfst-nolora-output/checkpoint-17500"
model_c = transformers.AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=1, load_in_8bit = True)
model_c.config.problem_type = "regression"
# model_c = cooking_time_model()
# model_c.to(device)
# def count_parameters(modelr):
#     return sum(p.numel() for p in modelr.parameters() if p.requires_grad)

# print(f'The model has {count_parameters(model_c):,} trainable parameters')

In [38]:
len(encodings)

3

In [6]:
model_c.eval()
with torch.no_grad():
    output = model_c(input_ids=ret[0],attention_mask=ret[1],token_type_ids=ret[2])

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [22]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min

In [23]:
model_best, _, _, l1 = load_ckp("best_model_a3.pt", model_c, optimizer)
model_current, _, _, l2 = load_ckp("current_checkpoint_a3.pt", model_c, optimizer)
print(l1==l2)
print(l1, l2)

True
0.00022455735384061483 0.00022455735384061483


In [11]:
def submission(dataloader, model_selected): # Code for submission
    preds = []
    targets = []
    model_selected.eval()
    with torch.no_grad():
        for batch_idx, (id, mask, tokentype, target) in enumerate(dataloader):
            outputs = model_selected(id, mask, tokentype)
            preds.append(list(outputs.logits))
            targets.append(list(target))
            if (batch_idx+1)%50 == 0:
                print("~"*15,f"{batch_idx+1}/{len(dataloader)}({(batch_idx+1)/len(dataloader)})","~"*15, end="\r")
            
    return preds, targets
ret, tar = submission(test_loader, model_best)

In [12]:
ltar = [l[0].tolist() for l in tar]
lret = [l[0].tolist()[0] for l in ret]
ltar = [round(l, 4) for l in ltar]
lret = [round(l, 4) if l>=0 else 0 for l in lret]
# lret = [l[0] for l in ret]

In [13]:
# for p, r in zip(ltar, lret):
#     # print(p, r)
#     print(abs(p-r)*100)
err = [abs(p-r) for p,r in zip(ltar,lret)]
print(np.mean(err))

0.006316959055973027


In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((0, 1),clip=True)
scaler.fit([[0], [100]])
err_min = scaler.inverse_transform([[i] for i in err])
# for m, e in zip(err_min, err):
#     print(m[0], e)
print("error stats:")
print("average:", np.mean(err_min))
print("max:", *err_min[np.argmax(err_min)])
print("max error target(minutes)",(scaler.inverse_transform([[data.iloc[np.argmax(err_min)][1]]])[0][0]))
print("max error prediction(minutes)",scaler.inverse_transform([[lret[np.argmax(err_min)]]])[0][0])
print("max error recipe:")
print(data.iloc[np.argmax(err_min)][0])
# data


error stats:
average: 0.6316959055973027
max: 92.7
max error target(minutes) 93.5
max error prediction(minutes) 0.8
max error recipe:
1 elephant, 2 rabbits (optional), 2 pails salt, 1 (8 oz.) canister black pepper, 3 bushels onions, <mask> gal. water, 6 pails flour


In [40]:
# test_input = ["<mask> lb. lean beef, cut across the grain into thin slices (you need a quick-cooking cut, such as minute steak), 1 lemongrass stalk, trimmed and finely chopped, <mask> tbsp. soy sauce, 2 tbs. fish sauce, 4 tsp. brown sugar, 0.5 tsp. chilli flakes, 1 lime, juiced, 3 tbsp. vegetable oil, 1 green pepper, thinly sliced, 2 bunches of spring onions, green and white parts separated and finely sliced, 6 garlic cloves, finely chopped, 1 tbsp. grated ginger, small bunch of basil, or purple basil, leaves picked and roughly chopped, cooked rice (about 250g uncooked weight), or cooked rice noodles, 50g roasted peanuts, roughly chopped"]
test_input = ['6 slices bacon, chopped, 0.5 c. chopped onion, <mask> c. chopped celery, 0.5 c. chopped green bell pepper, 3 cloves garlic, minced, 0.5 tsp. kosher salt, 0.5 tsp. freshly ground black pepper, 0.5 tsp. dried thyme, 0.25 tsp. cayenne pepper, 2 (12 ounce) packages frozen black-eyed peas, 4 c. reduced-sodium chicken broth, 2 c. cooked long grain rice, 0.25 c. chopped green onions, or as needed']
# test_input = ['<mask> lb. lean beef, cut across the grain into thin slices (you need a quick-cooking cut, such as minute steak), 1 lemongrass stalk, trimmed and finely chopped, <mask> tbsp. soy sauce, 2 tbsp. fish sauce, 4 tsp. brown sugar, 0.5 tsp. chili flakes, 1 lime, juiced, 3 tbsp. vegetable oil, 1 green pepper, thinly sliced, 2 bunches of spring onions, green and white parts separated and finely sliced, 6 garlic cloves, finely chopped, 1 tbsp. grated ginger, small bunch of basil, or purple basil, leaves picked and roughly chopped, cooked rice (about 250g uncooked weight), or cooked rice noodles, 50g roasted peanuts, roughly chopped']
test_input = ["0.25 c. unsalted butter, 0.25 c. all-purpose flour, 1 medium onion, chopped, 1 c. chopped celery, 0.67 c. chopped red bell pepper, 0.67 c. chopped green bell pepper, 2 cloves garlic, minced, 2 tsp. Cajun seasoning, 0.5 c. chicken broth, 0.5 lb. thinly sliced, fully cooked smoked sausage, 0.25 tsp. crushed red pepper flakes, 0.5 tsp. freshly ground black pepper, salt to taste (see Cook's Notes), 0.5 lb. raw shrimp, peeled and deveined (see Cook's Notes), 0.5 c. heavy whipping cream, <mask> oz. spaghetti, cooked, chopped fresh parsley for garnish (optional), lemon slices for garnish (optional)"]
def test_pipeline(test_input):
    single_df = pd.DataFrame({"ingr":test_input, "tags":[0.0]})
    # single_df
    s_test_customset = CustomDataset(single_df, tokenizer, MAX_LEN)
    s_test_loader = DataLoader(s_test_customset, batch_size=1, shuffle=False, collate_fn=colbatch)
    with torch.no_grad():
        for _, (id, mask, tokentype, _) in enumerate(s_test_loader):
            outputs = model_best(id, mask, tokentype)
            print(outputs.logits.tolist()[0][0])
            print(scaler.inverse_transform(outputs.logits.tolist())[0][0])
            return(outputs.logits.tolist()[0][0])
            
            # return outputs.logits.tolist(), scaler.inverse_transform(outputs.logits.tolist())
test_pipeline(test_input)
scaler.transform([[12]])

0.08980672806501389
8.980672806501389


array([[0.12]])